In [1]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx


class MeanSquaredError:
    def __init__(self):
        self.loss = None  # 손실 값
        self.y = None     # 예측값
        self.t = None     # 정답값

    def forward(self, y, t):
        self.y = y
        self.t = t
        # 오차 계산
        self.loss = 0.5 * np.mean((self.y - self.t) ** 2)
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) * dout / batch_size
        return dx

In [2]:
import numpy as np
z = np.array([1, 1])
z = np.reshape(z, (1,2))
print(z)
print(np.shape(z))

[[1 1]]
(1, 2)


In [7]:
np.random.seed(42)

W = np.random.randn(2,1)
print(np.shape(W))
print(W)
b = np.zeros(1, )
print(np.shape(b))
print(b)
affine = Affine(W, b)

(2, 1)
[[ 0.49671415]
 [-0.1382643 ]]
(1,)
[0.]


In [9]:
out = affine.forward(z)
print(out)
print(np.shape(out))

[[0.35844985]]
(1, 1)


In [10]:
dout = np.ones(1, )
print(dout)
print(np.shape(dout))

[1.]
(1,)


In [11]:
dout = affine.backward(dout)
print(dout)

[[ 0.49671415 -0.1382643 ]]


In [12]:
print(affine.dW)

[1. 1.]


In [13]:
t = np.zeros(1)
print(t)

[0.]


In [14]:
mse = MeanSquaredError()

In [15]:
out = mse.forward(out, t)
print(out)

0.06424314814207618


In [16]:
dout = np.ones(1)
dout = mse.backward(dout)
print(dout)

[[0.35844985]]


In [17]:
dout = affine.backward(dout)
print("손실함수에 대한 입력 데이터 z의 미분  :",dout)

손실함수에 대한 입력 데이터 z의 미분  : [[ 0.17804711 -0.04956082]]


In [18]:
dW = affine.dW
print(dW)

[[0.35844985]
 [0.35844985]]


In [19]:
db = affine.db
print(db)

[0.35844985]


In [21]:
lr = 0.01
affine.W = affine.W - lr * affine.dW
affine.b = affine.b - lr * affine.db

print(affine.W)
print(affine.b)

[[ 0.49312965]
 [-0.1418488 ]]
[-0.0035845]


In [22]:
out = affine.forward(z)
print(out)

[[0.34769636]]


In [23]:
out = mse.forward(out, t)
print("1번 경사하강법으로 이동한 후에 예측 값 : ", out)

1번 경사하강법으로 이동한 후에 예측 값 :  0.060446378086879485


In [24]:
print("0번 경사하강법으로 이동한 후에 예측 값 : ", 0.06424314814207618)
print("1번 경사하강법으로 이동한 후에 예측 값 : ", out)

0번 경사하강법으로 이동한 후에 예측 값 :  0.06424314814207618
1번 경사하강법으로 이동한 후에 예측 값 :  0.060446378086879485


### 최종 코드 : XOR 1번째 데이터 [1, 1]에 대한 정답값 [0]을 맞추는 모델 학습

In [30]:
# 입력 데이터와 정답값
z = np.array([1,1])
z = np.reshape(z, (1,-1))
t = np.zeros(1)
print(np.shape(z))
print(np.shape(t))

# 학습률
lr = 0.01

# 모델과 손실함수 설계
W = np.random.randn(2,1) # 2행1열에 주의 : 입력 데이터가 1행 2열로 들어오므로 (내적)
b = np.zeros(1)
affine = Affine(W, b)
mse = MeanSquaredError()

# 경사하강법 학습
for i in range(400):
    # 전파 Forward
    out = affine.forward(z)
    print("{} th 예측값 : {}".format(i, np.round(out, 3)))
    out = mse.forward(out, t)
    print("{} th 손실값 : {}".format(i, np.round(out, 3)))

    # 역전파 Backward
    dout = np.ones(1)
    dout = mse.backward(dout)
    dout = affine.backward(dout)

    # 경사하강법 Update
    affine.W = affine.W - lr * affine.dW
    affine.b = affine.b - lr * affine.db

(1, 2)
(1,)
0 th 예측값 : [[2.347]]
0 th 손실값 : 2.753
1 th 예측값 : [[2.276]]
1 th 손실값 : 2.591
2 th 예측값 : [[2.208]]
2 th 손실값 : 2.438
3 th 예측값 : [[2.142]]
3 th 손실값 : 2.293
4 th 예측값 : [[2.077]]
4 th 손실값 : 2.158
5 th 예측값 : [[2.015]]
5 th 손실값 : 2.03
6 th 예측값 : [[1.955]]
6 th 손실값 : 1.91
7 th 예측값 : [[1.896]]
7 th 손실값 : 1.798
8 th 예측값 : [[1.839]]
8 th 손실값 : 1.691
9 th 예측값 : [[1.784]]
9 th 손실값 : 1.591
10 th 예측값 : [[1.73]]
10 th 손실값 : 1.497
11 th 예측값 : [[1.679]]
11 th 손실값 : 1.409
12 th 예측값 : [[1.628]]
12 th 손실값 : 1.326
13 th 예측값 : [[1.579]]
13 th 손실값 : 1.247
14 th 예측값 : [[1.532]]
14 th 손실값 : 1.173
15 th 예측값 : [[1.486]]
15 th 손실값 : 1.104
16 th 예측값 : [[1.441]]
16 th 손실값 : 1.039
17 th 예측값 : [[1.398]]
17 th 손실값 : 0.977
18 th 예측값 : [[1.356]]
18 th 손실값 : 0.92
19 th 예측값 : [[1.316]]
19 th 손실값 : 0.865
20 th 예측값 : [[1.276]]
20 th 손실값 : 0.814
21 th 예측값 : [[1.238]]
21 th 손실값 : 0.766
22 th 예측값 : [[1.201]]
22 th 손실값 : 0.721
23 th 예측값 : [[1.165]]
23 th 손실값 : 0.678
24 th 예측값 : [[1.13]]
24 th 손실값 : 0.638
25 th 예측값 : [